## Problem 1

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

!pip install folium==0.5.0
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 79 kB 11.5 MB/s eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=a7079c10df13eb18bed99bb776b264e81f94f03a8f7928c7c2e5fc742561a4e6
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


 ##  Scraping the given Wikipedia page

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
print (soup.title)

table = str(soup.table)
table


<title>List of postal codes of Canada: M - Wikipedia</title>


'<table class="wikitable sortable">\n<tbody><tr>\n<th>Postal Code\n</th>\n<th>Borough\n</th>\n<th>Neighbourhood\n</th></tr>\n<tr>\n<td>M1A\n</td>\n<td>Not assigned\n</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M2A\n</td>\n<td>Not assigned\n</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M3A\n</td>\n<td>North York\n</td>\n<td>Parkwoods\n</td></tr>\n<tr>\n<td>M4A\n</td>\n<td>North York\n</td>\n<td>Victoria Village\n</td></tr>\n<tr>\n<td>M5A\n</td>\n<td>Downtown Toronto\n</td>\n<td>Regent Park, Harbourfront\n</td></tr>\n<tr>\n<td>M6A\n</td>\n<td>North York\n</td>\n<td>Lawrence Manor, Lawrence Heights\n</td></tr>\n<tr>\n<td>M7A\n</td>\n<td>Downtown Toronto\n</td>\n<td>Queen\'s Park, Ontario Provincial Government\n</td></tr>\n<tr>\n<td>M8A\n</td>\n<td>Not assigned\n</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M9A\n</td>\n<td>Etobicoke\n</td>\n<td>Islington Avenue, Humber Valley Village\n</td></tr>\n<tr>\n<td>M1B\n</td>\n<td>Scarborough\n</td>\n<td>Malvern, Rouge\n</td></tr>\n<tr>\n<td>M2

### Transforming the data into a pandas dataframe

In [14]:

dfs = pd.read_html(table)
df=dfs[0]
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


## Creating a dataframe as follows:
### 1. with three columns only:  PostalCode, Borough and Neighbourhood
### 2. ignoring cells with Borough = Not Assigned
### 3. combining two rows with the same neighbourhood
### 4. assigning the same neighbour as a borough when neighbourhood = Not assigned

In [16]:
df1 = df[df.Borough != 'Not assigned']
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Printing the shape of the dataframe

In [5]:
df2.shape

(103, 3)